In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score, cross_validate
# Deep Learning
import tensorflow as tf

In [69]:
def add_data_info():
    df1 = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-from-2017.csv', encoding= 'unicode_escape', low_memory=False)
    df2 = pd.read_csv('raw_data/charting-m-matches.csv', encoding= 'unicode_escape', low_memory=False)
    
    #get separated values from m-matches list of games - based on matchid
    df = pd.merge(df1,df2[['match_id','Date','Tournament','Round','Surface','Player 1','Player 2']], on=['match_id'], how='left')
    
    #add the condition of "Grand Slam" - 5 or 3 sets
    conditions = df['Tournament'].isin(["Wimbledon", "Australian Open", "US Open", "Roland Garros"])
    values = [1,0]

    # Create a new column  based on the conditions and values
    df['Grand Slam'] = pd.Series(pd.NA)
    df.loc[conditions, 'Grand Slam'] = values[0]
    df.loc[~conditions, 'Grand Slam'] = values[1]
    
    # save the original dataset with enriched data for future features
    df.to_csv('raw_data/charting-m-points-from-2017-enriched.csv', index=False)

In [70]:
def filter_player(df,opponent):
    
    # Read the dataset into a pandas DataFrame - from enriched file
    df = pd.read_csv(df, encoding= 'unicode_escape', low_memory=False)
    
    ## filtra pelo  oponente
    df = df[(df["Player 1"].str.contains(opponent)) | (df["Player 2"].str.contains(opponent))].copy()
    
    ##define se está sacando e se ganhou o ponto
    df['i_serve'] = np.where(((df['Svr'] == 1) & (df['Player 1'] == opponent )) | ((df['Svr'] == 2) & (df['Player 2'] == opponent)), 0, 1)
    df['i_win'] = np.where(((df['isSvrWinner'] == df['i_serve'])), 1, 0)
    df['is_second_service'] = np.where(((df['2nd'].isnull())), 0, 1)
    
    ## direção do saque
    # primeiro caractere do segundo saque
    df['dir_srv'] = df.loc[:, '2nd'].str[0]
    ## completa com o primeiro caractere do primeiro saque quando ele entrou
    df['dir_srv'].fillna(df.loc[:, '1st'].str[0], inplace=True)

    
    ##reseta o indice
    df.reset_index(inplace=True)
    
    ## salva arquivo e retorna dataset da funcao
    csv_file = "raw_data/" + opponent + "_points.csv"
    df.to_csv(csv_file, index=False)
    return df

In [71]:
add_data_info()

In [72]:
df = filter_player('raw_data/charting-m-points-from-2017-enriched.csv','Novak Djokovic')

In [73]:
df = df[df['dir_srv'].isin(['4','5','6'])]
df['rallyCount']=pd.to_numeric(df['rallyCount'])
df['dir_srv']=pd.to_numeric(df['dir_srv'])

In [90]:
features = ['Surface',
            'Round',
            'Grand Slam',
            'i_serve',
            'is_second_service',
            'dir_srv',
            'rallyCount',
            'isUnforced',
            'isForced'
            ]
X = df[features]
y = df['i_win']

In [91]:
feat_categorical = ['Surface','Round','dir_srv']

In [107]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18751 entries, 0 to 19323
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Surface            18751 non-null  object
 1   Round              18751 non-null  object
 2   Grand Slam         18751 non-null  int64 
 3   i_serve            18751 non-null  int64 
 4   is_second_service  18751 non-null  int64 
 5   dir_srv            18751 non-null  int64 
 6   rallyCount         18751 non-null  int64 
 7   isUnforced         18751 non-null  int64 
 8   isForced           18751 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 1.4+ MB


In [92]:
X['isUnforced'] = X['isUnforced'].replace({True: 1, False: 0})
X['isForced'] = X['isForced'].replace({True: 1, False: 0})

/var/folders/kk/0r0_hyfd673456ly0hptrbsc0000gn/T/ipykernel_3763/2793533693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['isUnforced'] = X['isUnforced'].replace({True: 1, False: 0})
/var/folders/kk/0r0_hyfd673456ly0hptrbsc0000gn/T/ipykernel_3763/2793533693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['isForced'] = X['isForced'].replace({True: 1, False: 0})


In [93]:
from sklearn.model_selection import train_test_split

# Train-Test Split
train, test = train_test_split(df, test_size=0.2)

# Train-Val Split
train, val = train_test_split(train, test_size=0.2)


In [94]:
## Separating features and target in the Train, Val and Test Set

X_train = train[features]
y_train = train['i_win']

X_val = val[features]
y_val = val['i_win']

X_test = test[features]
y_test = test['i_win']

In [95]:
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

12000 train examples
3000 validation examples
3751 test examples


In [96]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn import set_config
set_config(display="diagram")

In [97]:
preproc = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_exclude=["object"])),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), make_column_selector(dtype_include=["object"]))
)

preproc

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x17535a020>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1753586a0>)])

In [98]:
preproc.fit_transform(X_train).shape

/Users/alexandrealves/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(12000, 20)

In [99]:
X_train_preproc = preproc.fit_transform(X_train)
X_val_preproc = preproc.transform(X_val)
X_test_preproc = preproc.transform(X_test)

/Users/alexandrealves/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [100]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping

In [101]:
model = Sequential()

# 1. Architecture
model.add(layers.Dense(32, activation='relu', input_shape=(X_train_preproc.shape[-1],)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

# 2. Compiling
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [105]:
# 3. Training
es = EarlyStopping(monitor = 'val_accuracy',
                   patience = 5,
                   verbose = 0,
                   restore_best_weights = True)

history = model.fit(X_train_preproc,
                   y_train,
                   validation_data = (X_val_preproc, y_val),
                   batch_size = 64,
                   epochs = 500,
                   callbacks = [es], 
                   verbose = 1)

Epoch 1/500
188/188 [==============================] - 0s 768us/step - loss: 0.5196 - accuracy: 0.7025 - val_loss: 0.5092 - val_accuracy: 0.7050
Epoch 2/500
188/188 [==============================] - 0s 616us/step - loss: 0.5133 - accuracy: 0.7042 - val_loss: 0.5074 - val_accuracy: 0.7060
Epoch 3/500
188/188 [==============================] - 0s 607us/step - loss: 0.5106 - accuracy: 0.7044 - val_loss: 0.5068 - val_accuracy: 0.7100
Epoch 4/500
188/188 [==============================] - 0s 627us/step - loss: 0.5102 - accuracy: 0.7002 - val_loss: 0.5044 - val_accuracy: 0.7083
Epoch 5/500
188/188 [==============================] - 0s 606us/step - loss: 0.5139 - accuracy: 0.7042 - val_loss: 0.5032 - val_accuracy: 0.7093
Epoch 6/500
188/188 [==============================] - 0s 625us/step - loss: 0.5071 - accuracy: 0.7065 - val_loss: 0.5023 - val_accuracy: 0.7117
Epoch 7/500
188/188 [==============================] - 0s 711us/step - loss: 0.5062 - accuracy: 0.7053 - val_loss: 0.5011 - val_ac

In [106]:
res = model.evaluate(X_test_preproc, y_test)
print(f"accuracy = {res[1]:.3f}")

118/118 [==============================] - 0s 376us/step - loss: 0.4982 - accuracy: 0.7201
accuracy = 0.720


In [104]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 32)                672       
                                                                 
 dropout_14 (Dropout)        (None, 32)                0         
                                                                 
 dense_22 (Dense)            (None, 15)                495       
                                                                 
 dropout_15 (Dropout)        (None, 15)                0         
                                                                 
 dense_23 (Dense)            (None, 1)                 16        
                                                                 
Total params: 1,183
Trainable params: 1,183
Non-trainable params: 0
_________________________________________________________________
